# Scientific Research Project 7 - Practical 1

## Plotting the USGS earthquake dataset on a map
In the first exercise, we downloaded a CSV file of earthquake information from the USGS database. In this practical, we're going to use the Python programming language to make some nice map plots of the dataset.

To do that, we're going to use what's called a *Jupyter notebook*: an interactive notebook which contains a mixture of text and code blocks. To run the code in a particular cell, you simply click on that cell and either click the `Run` button in the top left, or press `Shift+Enter`.

In this practical, read through the contents of each cell and then run it to see what it does. **Make sure you understand what the code is doing - don't just blindly click through!** For your final project assignment, you can re-run and modify this notebook with your own dataset to make some nice plots for your report.
****

Ok, let's get started with some Python! Python is a very easy programming language which has a lot of built in packages that can help us to do analyses. To access these helpful pieces of code, we first of all have to `import` them. To do this we just use the `import` command as shown below. The main ones are:

* `numpy`: a very useful package which allows us to do some important mathematical computations. See here for more details: https://numpy.org/
* `pandas`: this package allows us to load in CSV files to what are called _dataframes_: you can think of a dataframe as being similar to a spreadsheet, but a lot more efficient. https://pandas.pydata.org/docs/getting_started/overview.html
* `matplotlib`: we use this to make nice figures and plots. You can look at the matplotlib gallery for examples of plots you can make with it: https://matplotlib.org/3.1.1/gallery/index.html